# Extraction of Down States from Neocortex recordings

Restarting from LFPwake0 and LFPwakeremoved.

LFPwakeremoved will be used to determined signal variance for threshold adjustement. 

LFPwake0 will be used for time determination. 

## Load LFP and packages

In [1]:
cd "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/"

\\10.69.168.1\crnldata\waking\audrey_hay\L1imaging\AnalysedMarch2023


In [2]:
from scipy import signal
from scipy import stats
from scipy.signal import find_peaks, peak_widths, hilbert
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
from scipy import fftpack
import pandas as pd
from pathlib import Path
import os
%matplotlib widget

folder_base = Path('Gaelle/Baseline_recording_ABmodified/Purple/session1/OpenEphys') 

filename = folder_base / f'LFPwake0.npy'
filename3 = folder_base / f'LFPwakeremoved.npy'
filename2 = folder_base / f'RawDataChannelExtractedDS.npy'
EMGbooleaninput = folder_base / f'EMGframeBoolean.pkl'
Channels = 'LFPChannels_perMice.xlsx' 

EMGboolean = pd.read_pickle(EMGbooleaninput)
LFPwakeremoved = np.load(filename3, mmap_mode= 'r')
All = np.load(filename2, mmap_mode= 'r')

def readfile(file):
  with open(file, encoding="utf-8") as file:
    list = []
    for row in file:
      temp = row.split()
      list.append(temp[:])
    return list

mice = os.path.basename(os.path.dirname(os.path.dirname(folder_base)))
allchannels = pd.read_excel(Channels)
EMGch=int(allchannels[mice][3])

CA1ch1=int(allchannels[mice][2].split(',')[0])
CA1ch2=int(allchannels[mice][2].split(',')[1])

EMG  =  All[:, EMGch]

CA1  =  All[:, CA1ch1]-All[:, CA1ch2] 
CA1wakeremoved = LFPwakeremoved[:,CA1ch1]-LFPwakeremoved[:,CA1ch2] 

CA1wakeremoved = CA1wakeremoved

## Continuous Wavelet Transform and projection calculation

First on signal with no wake time to determine sd of signal

In [8]:
# Filter parameter :
f_lowcut = 5
f_hicut = 9
N = 4
fs = 1000
nyq = 0.5 * fs
Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction

# Filter creation :
b, a = signal.butter(N, Wn, 'band')

filt_CA1 = signal.filtfilt(b, a, CA1)
filt_CA1wakeremoved = signal.filtfilt(b, a, CA1wakeremoved)

In [28]:
# Parameter and computation of CWT
w = 30.
freq = np.linspace(5, 9, 4)#18)
widths = w*fs / (2*freq*np.pi)
CA1NWcwt = signal.cwt(filt_CA1wakeremoved, signal.morlet2, widths, w=w)

CA1NWcwt = signal.cwt(filt_CA1, signal.morlet2, widths, w=w)

# Projection calculation CA1
absCA1NWcwt = np.absolute(CA1NWcwt)

zabsThetaCWT = stats.zscore(absCA1NWcwt, axis=None)

proj_ThetaCWT = np.max(zabsThetaCWT, axis = 0)/8

sdproj_ThetaCWT = np.std(proj_ThetaCWT)
meanproj_ThetaCWT = np.mean(proj_ThetaCWT)

numpnts = EMG.size
ThetaStatus = np.zeros(numpnts)
for ind in range(numpnts):
    if proj_ThetaCWT[ind]>(meanproj_ThetaCWT+1.4*sdproj_ThetaCWT):
        ThetaStatus[ind] = 1
        

proj_CA1NWcwt = np.sum(absCA1NWcwt, axis = 0)/24
sdproj_CA1cwt = np.std(proj_CA1NWcwt)
sd6proj_CA1cwt = sdproj_CA1cwt*6
sd7proj_CA1cwt = sdproj_CA1cwt*7

In [10]:
#####################################
########         CA1         #########
#####################################
# Conservative boolean filtering of CA1 filtered signal
BooleanCons = EMGboolean['BooleanConservative']
fCA1wake0C = filt_CA1.copy()
fCA1wake0C[BooleanCons] = 0
CA1wake0C = CA1.copy()
CA1wake0C[BooleanCons] = 0
# Liberal boolean filtering of CA1 filtered signal
BooleanLib = EMGboolean['BooleanLiberal']
fCA1wake0L = filt_CA1.copy()
fCA1wake0L[BooleanLib] = 0
CA1wake0L = CA1.copy()
CA1wake0L[BooleanLib] = 0

# Computation of CWT
CA1cwtWake0cons = signal.cwt(fCA1wake0C, signal.morlet2, widths, w=w)
CA1cwtWake0lib = signal.cwt(fCA1wake0L, signal.morlet2, widths, w=w)

# Projection calculation
absCA1W0Ccwt = np.absolute(CA1cwtWake0cons)
proj_CA1W0Ccwt = np.sum(absCA1W0Ccwt, axis = 0)/24
absCA1W0Lcwt = np.absolute(CA1cwtWake0lib)
proj_CA1W0Lcwt = np.sum(absCA1W0Lcwt, axis = 0)/24


In [11]:
from scipy.signal import find_peaks
from scipy.signal import chirp, find_peaks, peak_widths

# 7 sd threshold
peaks, properties = find_peaks(proj_CA1W0Lcwt, width=200, height=sd7proj_CA1cwt)
properties["prominences"], properties["widths"]

# Thetas boundaries taken at 70% from peak of intensity. This means that the Thetas with small amplitude will be longer than the big ones.
results_width = peak_widths(proj_CA1W0Lcwt, peaks, rel_height=0.7)

# Organise results in numpy array
peaks2 = peaks.reshape(len(peaks),1)
npresults_width = np.array(results_width).reshape(4,-1)
Theta_prop = np.append(peaks2, results_width).reshape(5,len(peaks2)).round()

In [12]:
projMaxP_cwtmg = np.max(CA1cwtWake0lib, axis = 0)
projMaxF_cwtmg = np.argmax(CA1cwtWake0lib, axis = 0)+ 10 #/2 + 8
projMaxP_cwtmg.shape

nb_Theta = np.arange(0,len(peaks),1)
data = np.zeros((len(peaks),4))

for tt in nb_Theta:
    Theta_start = int(Theta_prop[3,tt])
    Theta_stop = int(Theta_prop[4,tt])
    Theta_MaxP = projMaxP_cwtmg[Theta_start:Theta_stop]
    Theta_MaxF = projMaxF_cwtmg[Theta_start:Theta_stop]
    data[tt, 0] = max(Theta_MaxF).round()
    data[tt, 1] = max(Theta_MaxP).round()
    data[tt, 2] = round(sum(Theta_MaxF)/len(Theta_MaxF))
    data[tt, 3] = round(sum(Theta_MaxP)/len(Theta_MaxP))

param_Theta = pd.DataFrame(data, columns = ['Max freq', 'Max int', 'Avg freq', 'Avg int'])
tTheta_prop = Theta_prop.transpose()
pd_prop_Theta = pd.DataFrame(tTheta_prop, columns = ['peak time', 'Duration', 'peak amp', 'start time', 'end time'])
All_Theta = pd.concat([pd_prop_Theta, param_Theta], axis=1)

C:\Users\Manip2\AppData\Local\Temp\ipykernel_4756\1408940001.py:14: ComplexWarning: Casting complex values to real discards the imaginary part
  data[tt, 1] = max(Theta_MaxP).round()
C:\Users\Manip2\AppData\Local\Temp\ipykernel_4756\1408940001.py:16: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  data[tt, 3] = round(sum(Theta_MaxP)/len(Theta_MaxP))
C:\Users\Manip2\AppData\Local\Temp\ipykernel_4756\1408940001.py:16: ComplexWarning: Casting complex values to real discards the imaginary part
  data[tt, 3] = round(sum(Theta_MaxP)/len(Theta_MaxP))


In [13]:
nb_Theta = All_Theta.shape[0]
listtodrop = []
for tt in range(nb_Theta-1):
    if(All_Theta['end time'][tt]>All_Theta['start time'][tt + 1]):
        if(All_Theta['Duration'][tt]<All_Theta['Duration'][tt + 1]):
            if(All_Theta['start time'][tt]<All_Theta['start time'][tt + 1]):
                All_Theta['start time'][tt+1] = All_Theta['start time'][tt]
                listtodrop.append(tt)
            else:
                listtodrop.append(tt)
        if(All_Theta['Duration'][tt]>All_Theta['Duration'][tt + 1]):
            if(All_Theta['end time'][tt]<All_Theta['end time'][tt + 1]):
                All_Theta['end time'][tt] = All_Theta['end time'][tt + 1]
                listtodrop.append(tt+1)
            else:
                listtodrop.append(tt+1)

for tt in range(nb_Theta-1):
    if((All_Theta['start time'][tt + 1] - All_Theta['end time'][tt])<200):
        if((All_Theta['Duration'][tt]+300)<All_Theta['Duration'][tt + 1]):
            All_Theta['start time'][tt + 1] = All_Theta['start time'][tt]
            listtodrop.append(tt)
        if((All_Theta['Duration'][tt+1]+300)<All_Theta['Duration'][tt]):
            All_Theta['end time'][tt] = All_Theta['start time'][tt + 1]
            listtodrop.append(tt+1)

for tt in range(nb_Theta):
    All_Theta['Duration'][tt]=All_Theta['end time'][tt]-All_Theta['start time'][tt]
All_Theta = All_Theta.drop(listtodrop) 
#print(listtodrop)
All_Theta.shape[0]

3

### Display

ephys viewer to check Theta detection

In [20]:
EMG

memmap([  60.36229135,  -32.28840421, -126.12333706, ...,  319.76944527,
         198.54797105,  139.23883213])

In [31]:
from ephyviewer import mkQApp, MainViewer, TraceViewer
from ephyviewer import AnalogSignalSourceWithScatter
import ephyviewer

app = mkQApp()

sample_rate = 1000.
t_start = 0.

combined = np.stack([CA1, filt_CA1, proj_CA1W0Ccwt, proj_CA1W0Lcwt/1000, EMG/5000], axis = 1)

combined = np.stack([ThetaStatus], axis = 1)

Theta_start = Theta_prop[3,:].astype(int)
Theta_end = Theta_prop[4,:].astype(int)
Theta_start = All_Theta['start time'].to_numpy().astype(int)
Theta_end = All_Theta['end time'].to_numpy().astype(int)


#create 2 familly scatters from theses 2 indexes
scatter_indexes = {0: Theta_start, 1: Theta_end}
#and asign them to some channels each
scatter_channels = {0: [0, 1], 1: [0, 1]}
source = AnalogSignalSourceWithScatter(combined, sample_rate, t_start, scatter_indexes, scatter_channels)

#Create the main window that can contain several viewers
win = MainViewer(debug=True, show_auto_scale=True)

#create a viewer for signal with TraceViewer
#connected to the signal source
view1 = TraceViewer(source=source)

#Parameters can be set in script
#view1.params['scale_mode'] = 'same_for_all'
view1.params['display_labels'] = True
#And also parameters for each channel


view1.by_channel_params['ch0', 'color'] = '#ffffff'
view1.by_channel_params['ch1', 'color'] = '#0055ff'
view1.by_channel_params['ch0', 'gain'] = 0.0003
view1.by_channel_params['ch1', 'gain'] = 0.001
view1.by_channel_params['ch0', 'offset'] = 1
view1.by_channel_params['ch1', 'offset'] = -1


#put this viewer in the main window
win.add_view(view1)

#Run
win.show()
app.exec_()

: 